#### [문자 단위 RNN(Char RNN)]

- RNN의 입출력 단위 : 문자 레벨(character-level)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [3]:
char_set = list(set(sentence)) # 중복을 제거한 문자 집합 생성
char_dic = {c: i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩

In [4]:
print(char_dic)

{'f': 0, 'y': 1, ' ': 2, 'n': 3, 'c': 4, 'e': 5, 'm': 6, 'l': 7, '.': 8, 'd': 9, 'o': 10, 'p': 11, ',': 12, 's': 13, 'w': 14, 'a': 15, 'b': 16, 'u': 17, 'k': 18, 'h': 19, 't': 20, 'i': 21, 'r': 22, 'g': 23, "'": 24}


In [5]:
dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))

문자 집합의 크기 : 25


In [6]:
# 하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10  # 임의 숫자 지정
learning_rate = 0.1

In [7]:
# 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [8]:
print(x_data[0])   # if you wan
print(y_data[0])   # f you wan 

[21, 0, 2, 1, 10, 17, 2, 14, 15, 3]
[0, 2, 1, 10, 17, 2, 14, 15, 3, 20]


In [9]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # x 데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_100240\4234781144.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:278.)
  X = torch.FloatTensor(x_one_hot)


In [10]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])


In [11]:
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

- [3] 모델 설계<hr>

In [12]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size는 dic_size와 같음.
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [13]:
net = Net(dic_size, hidden_size, 2) # 이번에는 층을 두 개 쌓습니다.

In [14]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)
outputs = net(X)


In [15]:
print(outputs.shape) # 3차원 텐서
print(outputs.view(-1, dic_size).shape) # 2차원 텐서로 변환.

torch.Size([170, 10, 25])
torch.Size([1700, 25])


In [16]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [17]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    print(predict_str)

..fslf'.fse's'''f..fssf'.ff's...'ffs.ffff.''f.f'sfflssefs.fflf'lffsff'..'''s..'s..'ffs..fs.'fffsf'f'.fff..fs.f.fssf''s.fes.ffsf.f''f'sf'lf.f..ss'.fffefs..ffs.fs.f'..ff's.f''fff'sf
                                       t  t      t        t                   t            t    t     t   t         t      t     t    t           t   t                        t   
                    t t           t           e                                     t                           t   t                 t                                            
ootodtoododtodoo odotodododododododododtodtododdotododotodtodtodtooootodootodtdododododooottdtodtodoootododododododododododtododdtododtodto ootodotododododotodododotodtoodtodododo
nfb.yes.odh..s.yb ,ybnsks..ea.s..yb.s..s.y..s.ee...s.yy..s.hdeon.,y.ne.se.s..yayb.y's.s..ydtoeby..s..y.oseshby..b.yebh.ye.y.os.oyb,e....d..y..s.s byb....hd..y..d hhby.eb.ythe...sn
shehsethhe,eopeosb,eeoseeeopeheetoeoeeobbeeeepeeepoeeeeeeneoeehse,epethnepthsptheetoefseah'eheeotohb

poaorltont to luil, asseip, bon'thdont upepeo le to ether to lo lecthtong ant aon'thdssign them tosks ant aon , bui rosher touch the  to lon' tor the tnd e s iod nsith ao the cosc
poaon tont to cutl, asship, don't dont upeppople to echer to lo lect tong ant aon'thdssign them tosks ant aon , but aosher tosch them to lon' for themtnd e s ip  nsith an the chsc
pnaon tont to cuild a ship, don't dont rp people th ether th co le t tong ant don't assign them tes s tnd aon , bui a sher tosch them to lon' tor them nd e s im  nsith fn the shsc
pnaon tont wo luild a ship, don't do t rp people to ether th lonle t wong and don't dssign them tos s and won , bui ansher toach them to long tor them nd e s im  nsit  on the shds
pnaorltont do luild a ship, don't do m ap  eople to ether to bonlect wong and don't dssign the  tas s and won , but ansher tosch them to bon' tor the  ndless imu nsit  of the shdc
pnaorltont do build a shipe don't drum rp people to ether th lonlect wond and don't assign them tosi

In [18]:
rnn = nn.RNN(10, 20, 2)

input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)

output, hn = rnn(input, h0)
